# Tide Gauge Data Retrieval and Processing

This notebook introduces an example of a workflow one might encounter when processing tide gauge data. These days raw water level data are being processed by corresponding distribution centers and packed into a netCDF4 format among others. Since it's most 'native' for opening with Xarray this is what will be demonstrated. \
Data have been downloaded from [NOAA Tides and Currents Server](https://tidesandcurrents.noaa.gov/) and contain all the tide gauges data. \
For information on how to download the data see [here](https://api.tidesandcurrents.noaa.gov/api/prod/) 

### Learning Goals

* Basics of Data Exploration
* Basics of Xarray (understanding selecting, averaging, masking)
* Create Pandas Dataframe
* Vizualize geographically scattered data using hvplot
* Learn Xarray open_mfdataset preprocess option
* Learn to combine multiple files into one Xarray Dataset

## TG Processing (interpolation, selection)

In [1]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import hvplot.xarray
import cartopy.crs as ccrs      
import holoviews as hv
hv.extension('bokeh')

import netCDF4
import glob
import os
#%load_ext watermark

In [2]:
path = '/Users/ynorden/Research/code/coastal-sl/data/tide_gauges/'
filelist = sorted(glob.glob(os.path.join(path, '*.nc')))

In [3]:
filename = [os.path.basename(file) for file in filelist]

1. Let's look at a single file

In [4]:
station_1 = xr.open_dataset(path+'1890000.nc')
station_2 = xr.open_dataset(path+'2695535.nc')
print(station_1,station_2)

<xarray.Dataset>
Dimensions:    (time: 612920)
Coordinates:
  * time       (time) datetime64[ns] 1950-01-01T01:00:00 ... 2021-05-31T23:00:00
Data variables:
    observed   (time) float32 ...
    predicted  (time) float32 ...
Attributes: (12/21)
    id:                    1890000
    name:                  Wake Island, Pacific Ocean
    state:                 United States of America
    latitude:              19.290556
    longitude:             166.6175
    timezone:              NZST
    ...                    ...
    harmonicConstituents:  https://api.tidesandcurrents.noaa.gov/mdapi/prod/w...
    benchmarks:            https://api.tidesandcurrents.noaa.gov/mdapi/prod/w...
    tidePredOffsets:       https://api.tidesandcurrents.noaa.gov/mdapi/prod/w...
    products:              https://api.tidesandcurrents.noaa.gov/mdapi/prod/w...
    disclaimers:           https://api.tidesandcurrents.noaa.gov/mdapi/prod/w...
    notices:               https://api.tidesandcurrents.noaa.gov/mdapi/pr

What immediately stands out is how many attributes (metadata) there is. Also each tige dauge is a time-series with a different length. \
A logical question when trying to explore these data is:\
**What is the best way to combine these data into a single dataset?**


In [ ]:
mfds = xr.open_mfdataset(filelist)

### Reading metadata into a DataFrame

It makes sense to read all the relevant metadata into a dictionary first 

In [5]:
#filename

In [6]:
lats = []
lons = []
number = []
city = []
start_times = []
end_times = []
for fname in filelist:
    with xr.open_dataset(fname) as ds:
        lats.append(ds.latitude)
        lons.append(ds.longitude)
        number.append(ds.id)
        city.append(ds.name)
        start_times.append(ds.time.values[0])
        end_times.append(ds.time.values[-1])

In [7]:
d = {"id": filename, 
     "city": city,
     "latitude": lats,
     "longitude": lons,
     "start times": start_times,
     "end times": end_times
}

Then convert this dictionary into a DataFrame next.

In [8]:
df_meta = pd.DataFrame(d)
df_meta

,id,city,latitude,longitude,start times,end times
0,1611400.nc,Nawiliwili,21.954400,-159.356100,1954-01-01 01:00:00,2021-05-31 23:00:00
1,1612340.nc,Honolulu,21.306694,-157.867000,1914-01-01 01:00:00,2021-05-31 23:00:00
2,1612480.nc,Mokuoloe,21.433056,-157.790000,1981-01-01 01:00:00,2021-05-31 23:00:00
3,1615680.nc,"Kahului, Kahului Harbor",20.895000,-156.476694,1954-01-01 01:00:00,2021-05-31 23:00:00
4,1617433.nc,Kawaihae,20.036600,-155.829400,1990-01-01 01:00:00,2021-05-31 23:00:00
...,...,...,...,...,...,...
229,9755371.nc,"San Juan, La Puntilla, San Juan Bay",18.459167,-66.116389,1977-01-01 01:00:00,2021-05-31 23:00:00
230,9757809.nc,Arecibo,18.480528,-66.702361,2008-01-01 01:00:00,2017-12-31 23:00:00
231,9759110.nc,Magueyes Island,17.970100,-67.046402,1954-01-01 01:00:00,2021-05-31 23:00:00
232,9759394.nc,Mayaguez,18.219000,-67.162500,2008-01-01 01:00:00,2021-05-31 23:00:00


Here we have all the metadate (notice, that we have not read actual water levels yet).\
We have **hourly** water level for 234 stations with varyng record lengths. 

### Selecting data based on desired criteria

What question are we trying to answer? \
In this instance I'm interested in looing at tide gauges along the U.S. East Coast.\
The range of my longitudes is (-180,180) hence I need to select longitudes belonging to (-100,-50) inteval. \
\
Another part of my scientific request is I want my time-series have the same record length as the available satellite altimetry data hence I want it to start on 1993-01-01 and go as long as possible into present. 

In [9]:
east_coast = df_meta.loc[(df_meta.longitude < -50) & (df_meta.longitude > -100) &
                        (df_meta["start times"] <= '1993-01-01T01:00:00.000000000'
                          ) & (df_meta["end times"] >= '2019-04-15T01:00:00.000000000')]

In [10]:
east_coast

,id,city,latitude,longitude,start times,end times
11,2695535.nc,Bermuda Biological Station,32.369999,-64.695000,1968-01-01 01:00:00,2021-05-31 23:00:00
12,2695540.nc,"Bermuda, St. Georges Island",32.373306,-64.703306,1989-01-01 01:00:00,2019-11-23 17:00:00
13,8410140.nc,Eastport,44.904598,-66.982903,1929-01-01 01:00:00,2021-05-31 23:00:00
15,8413320.nc,Bar Harbor,44.392194,-68.204278,1947-01-01 01:00:00,2021-05-31 23:00:00
16,8418150.nc,Portland,43.658060,-70.244170,1910-01-01 01:00:00,2021-05-31 23:00:00
...,...,...,...,...,...,...
155,8779770.nc,Port Isabel,26.061167,-97.215528,1944-01-01 01:00:00,2021-05-31 23:00:00
224,9751401.nc,Lime Tree Bay,17.694700,-64.753799,1982-01-01 01:00:00,2021-05-31 23:00:00
225,9751639.nc,Charlotte Amalie,18.335830,-64.920000,1975-01-01 01:00:00,2021-05-31 23:00:00
229,9755371.nc,"San Juan, La Puntilla, San Juan Bay",18.459167,-66.116389,1977-01-01 01:00:00,2021-05-31 23:00:00


Now my DataFrame has 73 stations. \
Let's plot the data and see if the selection worked. \
`import hvplot.pandas` and `cartopy` are required

In [11]:
import hvplot.pandas
east_coast.hvplot.points('longitude','latitude',c='id',geo=True,projection=ccrs.PlateCarree(),
                                    coastline=True,line_width=5, title = 'Tige Gauges on the East Coast', hover_cols=['id','city'],legend=False)

:Overlay
   .Points.I    :Points   [longitude,latitude]   (id,city)
   .Coastline.I :Feature   [Longitude,Latitude]

**Success**!

### Using xarray.open_mfdataset( preprocess option)

We now have a preliminary list of stations that fir 'east coast' condition.\
My goals:
1. Important part of my setup is that tige gauge water level data come with two variables (see 'station_1' above): observed and predicted. Predicted variable gives tides that are normally well-known for a given location. Observed variable is actual measurements. \
I want to work with sea level anomalies (sla). in order to get them I need to subtract observed from predicted.
2. I also want to work with daily data. I'd like to choose days that have at least 18 hours of measurements to average over, otherwise replace a day with a NaN. 
3. I also want to have all my data in one Dataset. 

My time-series are scattered geographically and have different record length. \
How do I achieve goal 3 ? 

Xarray has a powerful option `preprocess` and in the cells below I will demonstrate how to use it. \
When looking at the data in dataframe format all columns seem equal, every column can become a variable. \
However, the very logic of Xarray Dataset is inviting to rethink the data we have in terms of their core dimentions. \
Our data are time-series so one core dimension is time.\
Because we want to be able to easily extract time-series from different stations another core dimension we need to introduce would be what we will call 'id' or 'station' or it could be 'n'. \
Dimensions are not to be confused with coordinates. \
So far 'id' associated with each station is more of a coordinate which is still important to keep. \
We need to begin the preprocess function by introducing a new dimention 'id' to each passing file, that's the link that will keep them together. 

In [12]:
def preprocess(tg_1):
    
    tg_1 = tg_1.expand_dims("id")
    
    tg_1.coords["id"] = xr.DataArray([tg_1.attrs['id']], dims="id")
    tg_1.coords["lat"] = xr.DataArray([tg_1.attrs['latitude']], dims="id")
    tg_1.coords["lon"] = xr.DataArray([tg_1.attrs['longitude']], dims="id")
    tg_1.coords["city"] = xr.DataArray([tg_1.attrs['name']], dims="id")
    tg_1.coords["state"] = xr.DataArray([tg_1.attrs['state']], dims="id")

    tg_1 = tg_1.sel(time=slice('1993-01-01','2019-10-15'))

    tg_1['sla']=tg_1.observed - tg_1.predicted

    resampler = tg_1.sla.resample(time="D")
    daily_mean = resampler.mean()
    counts = resampler.count(fill_value=0)
    
    newtime = xr.date_range("1993-01-01", "2019-10-15", freq="D")
    daily_mean = daily_mean.where(counts>=18)
    daily_mean = daily_mean.reindex(time=newtime)
    
    return daily_mean

? I can add a little chapter on what would happen if I simply try writing files into one dataset without 'preprocess' option. 

In [13]:
%time
ds_all = xr.open_mfdataset(path+east_coast.id.values, preprocess=preprocess, parallel=True, chunks={"time":-1}, combine="nested", concat_dim="id")
ds_all

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.1 µs


<xarray.DataArray 'sla' (id: 73, time: 9784)>
array([[        nan,         nan,         nan, ..., -0.05195833,
        -0.05770833, -0.03904166],
       [-0.170875  , -0.16329168, -0.24754167, ...,  0.02975   ,
         0.02995834,  0.045875  ],
       [-0.0025    , -0.21733336, -0.18304168, ...,  0.16099998,
         0.16966665,  0.16312496],
       ...,
       [        nan,         nan,         nan, ...,  0.10591667,
         0.13320833,  0.144375  ],
       [-0.02425   , -0.02004167, -0.008625  , ...,  0.10508334,
         0.10979167,  0.11654166],
       [-0.04383333, -0.05525   , -0.02575   , ...,  0.10841667,
         0.13058333,  0.15766667]], dtype=float32)
Coordinates:
  * id       (id) <U7 '2695535' '2695540' '8410140' ... '9755371' '9759110'
  * time     (time) datetime64[ns] 1993-01-01 1993-01-02 ... 2019-10-15
    lat      (id) float64 32.37 32.37 44.9 44.39 ... 17.69 18.34 18.46 17.97
    lon      (id) float64 -64.69 -64.7 -66.98 -68.2 ... -64.92 -66.12 -67.05
    city     (id) <U39 'Bermuda Biological Station' ... 'Magueyes Island'
    state    (id) <U7 '' 'Bermuda' 'ME' 'ME' 'ME' ... 'TX' 'VI' 'VI' 'PR' 'PR'

In [14]:
ds_all.sel(id='8770613')

<xarray.DataArray 'sla' (time: 9784)>
array([       nan,        nan,        nan, ..., 0.16766667, 0.29679167,
       0.29945835], dtype=float32)
Coordinates:
    id       <U7 '8770613'
  * time     (time) datetime64[ns] 1993-01-01 1993-01-02 ... 2019-10-15
    lat      float64 29.68
    lon      float64 -94.98
    city     <U39 'Morgans Point, Barbours Cut'
    state    <U7 'TX'

We now have 73 stations. Plot below shows that may still have quite significant gaps in the time-series. 

In [15]:
ds_all.sel(id='8534720').hvplot()

:Curve   [time]   (sla)

## Selecting time-series with 95% completeness. Interpolation.

I want to keep the stations that are at least 95% complete in terms of days of the total record length. \
Step 1. Create a mask

In [16]:
mask = (ds_all.count(dim='time')/ ds_all.time.size * 100)>=95
ds_tg  = ds_all.sel(id=mask).interpolate_na(dim='time',method='linear') #tg stands for tide gauges 

In [17]:
ds_all.sel(id='8770613').hvplot()

:Curve   [time]   (sla)

In [18]:
#(ds_tg.hvplot.scatter(x='lon', y='lat', c='id', geo=True,coastline=True)+ds_tg.isel(id='1').hvplot()).cols(1)

In [19]:
ds_tg.sel(id='8771450').hvplot()

:Curve   [time]   (sla)

In [20]:
ds_tg

<xarray.DataArray 'sla' (id: 46, time: 9784)>
array([[-0.0025    , -0.21733336, -0.18304168, ...,  0.16099998,
         0.16966665,  0.16312496],
       [-0.01983334, -0.20708334, -0.15291668, ...,  0.15699999,
         0.169125  ,  0.12766667],
       [ 0.00166669, -0.16950001, -0.10812501, ...,  0.20200002,
         0.22229166,  0.17383333],
       ...,
       [-0.052625  , -0.03920833, -0.01991667, ...,  0.14320834,
         0.16079167,  0.17329167],
       [-0.02425   , -0.02004167, -0.008625  , ...,  0.10508334,
         0.10979167,  0.11654166],
       [-0.04383333, -0.05525   , -0.02575   , ...,  0.10841667,
         0.13058333,  0.15766667]], dtype=float32)
Coordinates:
  * id       (id) <U7 '8410140' '8418150' '8443970' ... '9755371' '9759110'
  * time     (time) datetime64[ns] 1993-01-01 1993-01-02 ... 2019-10-15
    lat      (id) float64 44.9 43.66 42.35 41.52 ... 26.06 17.69 18.46 17.97
    lon      (id) float64 -66.98 -70.24 -71.05 -70.67 ... -64.75 -66.12 -67.05
    city     (id) <U39 'Eastport' 'Portland' ... 'Magueyes Island'
    state    (id) <U7 'ME' 'ME' 'MA' 'MA' 'MA' 'RI' ... 'TX' 'TX' 'VI' 'PR' 'PR'

In [21]:
drop_id = ('8774770','8723970', '8771013', '8770613',
           '8635750',
           '8594900','8574680','8575512','8571892','8551910','8545240',
           '8454000',
           '9759110','9755371','9751401')
# 8770613 is something I need to fix

In [22]:
ds_tg

<xarray.DataArray 'sla' (id: 46, time: 9784)>
array([[-0.0025    , -0.21733336, -0.18304168, ...,  0.16099998,
         0.16966665,  0.16312496],
       [-0.01983334, -0.20708334, -0.15291668, ...,  0.15699999,
         0.169125  ,  0.12766667],
       [ 0.00166669, -0.16950001, -0.10812501, ...,  0.20200002,
         0.22229166,  0.17383333],
       ...,
       [-0.052625  , -0.03920833, -0.01991667, ...,  0.14320834,
         0.16079167,  0.17329167],
       [-0.02425   , -0.02004167, -0.008625  , ...,  0.10508334,
         0.10979167,  0.11654166],
       [-0.04383333, -0.05525   , -0.02575   , ...,  0.10841667,
         0.13058333,  0.15766667]], dtype=float32)
Coordinates:
  * id       (id) <U7 '8410140' '8418150' '8443970' ... '9755371' '9759110'
  * time     (time) datetime64[ns] 1993-01-01 1993-01-02 ... 2019-10-15
    lat      (id) float64 44.9 43.66 42.35 41.52 ... 26.06 17.69 18.46 17.97
    lon      (id) float64 -66.98 -70.24 -71.05 -70.67 ... -64.75 -66.12 -67.05
    city     (id) <U39 'Eastport' 'Portland' ... 'Magueyes Island'
    state    (id) <U7 'ME' 'ME' 'MA' 'MA' 'MA' 'RI' ... 'TX' 'TX' 'VI' 'PR' 'PR'

In [23]:
ds_tg = ds_tg.drop_sel(id=drop_id)

In [24]:
ds_tg.hvplot.scatter(x='lon', y='lat', c='id',geo=True, coastline=True)

:DynamicMap   [time]
   :Overlay
      .Scatter.I   :Scatter   [lon]   (lat,id)
      .Coastline.I :Feature   [Longitude,Latitude]

In [25]:
ds_tg

<xarray.DataArray 'sla' (id: 31, time: 9784)>
array([[-0.0025    , -0.21733336, -0.18304168, ...,  0.16099998,
         0.16966665,  0.16312496],
       [-0.01983334, -0.20708334, -0.15291668, ...,  0.15699999,
         0.169125  ,  0.12766667],
       [ 0.00166669, -0.16950001, -0.10812501, ...,  0.20200002,
         0.22229166,  0.17383333],
       ...,
       [-0.02708334,  0.02545833,  0.17129166, ...,  0.28770834,
         0.37770832,  0.37991667],
       [ 0.037125  ,  0.13329168,  0.16391666, ...,  0.31658334,
         0.32137498,  0.301375  ],
       [ 0.02429166,  0.05679167,  0.11225   , ...,  0.37241665,
         0.33220834,  0.30066666]], dtype=float32)
Coordinates:
  * id       (id) <U7 '8410140' '8418150' '8443970' ... '8775870' '8779770'
  * time     (time) datetime64[ns] 1993-01-01 1993-01-02 ... 2019-10-15
    lat      (id) float64 44.9 43.66 42.35 41.52 ... 29.26 29.31 27.58 26.06
    lon      (id) float64 -66.98 -70.24 -71.05 -70.67 ... -94.79 -97.22 -97.22
    city     (id) <U39 'Eastport' 'Portland' ... 'Port Isabel'
    state    (id) <U7 'ME' 'ME' 'MA' 'MA' 'MA' 'RI' ... 'FL' 'LA' 'TX' 'TX' 'TX'

## Save dataset

In [26]:
#ds_tg.to_netcdf('/Users/ynorden/Research/code/coastal-sl/data/'+'ds_tg.nc')